In [1]:
# Import dependencies
from sleeperpy import User, Leagues
import pandas as pd

In [2]:
# User inputs username and selects a league from landing page
username = "dirtywizard"
# Get User's Sleeper ID and Sleeper leagues
user_id = User.get_user(username)['user_id']
leagues = Leagues.get_all_leagues(user_id, 'nfl', 2024)
league_name_list = [x['name'] for x in leagues]
# Show the user the list of names and have them select one
print(league_name_list)
league_name = "No Coffee Sharps D.F.F.L."
# Find the league id for the selected league
league_id = [x['league_id'] for x in leagues if x['name'] == league_name][0]

['No Coffee Sharps D.F.F.L.']


In [3]:
# Receive user ID & league ID in GET request, then fetch league data from Sleeper API
selected_league = Leagues.get_league(league_id)
roster_positions = selected_league['roster_positions']
scoring_settings = selected_league['scoring_settings']
# Get the rosters for the league
rosters = Leagues.get_rosters(league_id)
users = Leagues.get_users(league_id)
# Get a list of all players currently on rosters
rostered_players = [x['players'] for x in rosters]
rostered_players = [item for sublist in rostered_players for item in sublist]

In [4]:
# Read RDS database 
# Select id_sleeper, full_name, age, team, number, position from players where id_sleeper in rostered_players
allplayers = pd.read_csv('test_data/rds_players.csv', dtype={'id_sleeper': 'str'}) 
# Read RDS database
# Select * from predictions where id_sleeper in rostered_players
predictions = pd.read_csv('test_data/rds_predictions.csv', dtype={'id_sleeper': 'str'})
predictions = predictions.loc[predictions['id_sleeper'].isin(rostered_players)]
predictions = predictions.merge(allplayers[['id_sleeper', 'full_name', 'age', 'team', 'number', 'position']], on='id_sleeper', how='left')

# Multiply each prediction column by the corresponding scoring setting
for column in predictions.columns:
    if column in scoring_settings:
        predictions[column] = predictions[column] * scoring_settings[column]
# Sum up the total points for each column in the scoring settings
predictions['predicted_fantasy_points'] = predictions[list(scoring_settings.keys())].sum(axis=1)
predictions = predictions.drop(columns=list(scoring_settings.keys())).drop(columns=['index_predictions'])
predictions.sort_values(by='predicted_fantasy_points', ascending=False, inplace=True, ignore_index=True)
predictions = predictions[['id_sleeper', 'full_name', 'age', 'team', 'number', 'position', 'week_of_season', 'predicted_fantasy_points']]
predictions.head()

,id_sleeper,full_name,age,team,number,position,week_of_season,predicted_fantasy_points
0,4984,Josh Allen,28.0,BUF,17.0,QB,8,25.760
1,4984,Josh Allen,28.0,BUF,17.0,QB,1,25.760
2,4034,Christian McCaffrey,27.0,SF,23.0,RB,10,24.915
3,4034,Christian McCaffrey,27.0,SF,23.0,RB,5,24.915
4,4984,Josh Allen,28.0,BUF,17.0,QB,14,24.350


In [5]:
# Bring in the franchise_id for each roster
roster_df = pd.DataFrame(rosters)
roster_df = roster_df[['players', 'owner_id']]
# Explode the roster_df to get a row for each player on each roster
roster_df = roster_df.explode('players')
roster_df['players'] = roster_df['players'].astype(str)
roster_df = roster_df.rename(columns={'players': 'id_sleeper'})
roster_df.head()

,id_sleeper,owner_id
0,11575,208058775418961920
0,11643,208058775418961920
0,8139,208058775418961920
0,10871,208058775418961920
0,10235,208058775418961920


In [6]:
# Get info about each franchise
franchises = pd.DataFrame()
franchises['owner_id'] = [x['user_id'] for x in users]
franchises['owner_name'] = [x['display_name'] for x in users]
franchises['team_name'] = [x['metadata']['team_name'] if 'team_name' in x['metadata'] else x['display_name'] for x in users]
# Create a True/False column for whether the owner is the user
franchises['is_user'] = False
franchises.loc[franchises['owner_id'] == user_id, 'is_user'] = True
franchises

,owner_id,owner_name,team_name,is_user
0,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
1,342869377223385088,nrector,The Fort Worth Fremen,False
2,412722740429082624,jccarm,DangeRUSS Last Ride,False
3,452577139540094976,2014champ,2014champ,False
4,461323803130064896,cdulgar,Idk much about soccer,False
5,725907041763303424,dirtywizard,Croccity Body Snatchers,True
6,904381167934095360,mdwglobe,GusTheBus,False
7,904398822170636288,SpacemnSpiff,Saskatoon Squatches,False
8,904399374891855872,comebackking15,Comeback 👑’s,False
9,904400906127458304,TrulyUhle,Verdanks Vacqueros FFC,False


In [37]:
# Merge the predictions, roster_df, and franchises dataframes
franchise_comparison = predictions.merge(roster_df, on='id_sleeper', how='left')
franchise_comparison = franchise_comparison.merge(franchises, on='owner_id', how='left')
franchise_comparison = franchise_comparison.drop(columns=['owner_id'])
franchise_comparison.head()

,id_sleeper,full_name,age,team,number,position,week_of_season,predicted_fantasy_points,owner_name,team_name,is_user
0,4984,Josh Allen,28.0,BUF,17.0,QB,8,25.760,comebackking15,Comeback 👑’s,False
1,4984,Josh Allen,28.0,BUF,17.0,QB,1,25.760,comebackking15,Comeback 👑’s,False
2,4034,Christian McCaffrey,27.0,SF,23.0,RB,10,24.915,jccarm,DangeRUSS Last Ride,False
3,4034,Christian McCaffrey,27.0,SF,23.0,RB,5,24.915,jccarm,DangeRUSS Last Ride,False
4,4984,Josh Allen,28.0,BUF,17.0,QB,14,24.350,comebackking15,Comeback 👑’s,False


In [38]:
# Set a dictionary describing each of the roster positions
rosterDict = {
    'QB':['QB'], 
    'RB':['RB'],
    'WR':['WR'],
    'TE':['TE'],
    'K':['K'],
    'DEF':['DEF'],
    'FLEX':['RB', 'WR', 'TE'],
    'SUPER_FLEX':['QB', 'RB', 'WR', 'TE'],
}

# Mark players who would be started in the roster_position with a "starter" column, initialize as False
franchise_comparison['starter'] = False
# Loop through roster positions
for roster_position in roster_positions:
    # Check that the roster_position is not "BN"
    if roster_position != "BN":
        # Loop through each week of the season
        for week in franchise_comparison['week_of_season'].unique():
            # Loop through the franchises
            for franchise in franchise_comparison['owner_name'].unique():
                # Get the players for the current franchise, week, and roster position
                players = franchise_comparison[(franchise_comparison['owner_name'] == franchise) & (franchise_comparison['week_of_season'] == week) & (franchise_comparison['position'].isin(rosterDict[roster_position]))]
                # Check that the number of players is greater than 0
                if len(players) > 0:
                    # Get the player with the highest predicted fantasy points
                    max_player = players.loc[players['predicted_fantasy_points'].idxmax()]
                    # Mark the player as a starter
                    franchise_comparison.loc[max_player.name, 'starter'] = True

In [20]:
test = franchise_comparison.sort_values(by='predicted_fantasy_points', ascending=True).reset_index(drop=True)
print(test['predicted_fantasy_points'].idxmax())
test

6409


,id_sleeper,full_name,age,team,number,position,week_of_season,predicted_fantasy_points,owner_name,team_name,is_user,starter
0,SEA,Seattle Seahawks,NaN,SEA,NaN,DEF,14,-1.500,SpacemnSpiff,Saskatoon Squatches,False,False
1,IND,Indianapolis Colts,NaN,IND,NaN,DEF,1,-1.260,cdulgar,Idk much about soccer,False,False
2,11564,Drake Maye,21.0,NE,10.0,QB,7,-0.080,mdwglobe,GusTheBus,False,False
3,11564,Drake Maye,21.0,NE,10.0,QB,6,-0.080,mdwglobe,GusTheBus,False,False
4,11564,Drake Maye,21.0,NE,10.0,QB,1,-0.080,mdwglobe,GusTheBus,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
6406,4984,Josh Allen,28.0,BUF,17.0,QB,14,24.350,comebackking15,Comeback 👑’s,False,False
6407,4034,Christian McCaffrey,27.0,SF,23.0,RB,5,24.915,jccarm,DangeRUSS Last Ride,False,False
6408,4034,Christian McCaffrey,27.0,SF,23.0,RB,10,24.915,jccarm,DangeRUSS Last Ride,False,False
6409,4984,Josh Allen,28.0,BUF,17.0,QB,1,25.760,comebackking15,Comeback 👑’s,False,False


In [9]:
roster_positions

['QB',
 'RB',
 'RB',
 'WR',
 'WR',
 'WR',
 'TE',
 'TE',
 'FLEX',
 'FLEX',
 'SUPER_FLEX',
 'K',
 'DEF',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN']

In [8]:
franchise_comparison.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6411 entries, 0 to 6410
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id_sleeper                6411 non-null   object 
 1   full_name                 6411 non-null   object 
 2   age                       5935 non-null   float64
 3   team                      6411 non-null   object 
 4   number                    5901 non-null   float64
 5   position                  6411 non-null   object 
 6   week_of_season            6411 non-null   int64  
 7   predicted_fantasy_points  6411 non-null   float64
 8   owner_name                6411 non-null   object 
 9   team_name                 6411 non-null   object 
 10  is_user                   6411 non-null   bool   
dtypes: bool(1), float64(3), int64(1), object(6)
memory usage: 507.2+ KB


In [52]:
# Convert the dataframe to json
franchise_comparison_json = franchise_comparison.to_json(orient='records')

'[{"id_sleeper":"4984","full_name":"Josh Allen","age":28.0,"team":"BUF","number":17.0,"position":"QB","week_of_season":8,"predicted_fantasy_points":25.7600000092,"owner_name":"comebackking15","team_name":"Comeback \\ud83d\\udc51\\u2019s","is_user":false},{"id_sleeper":"4984","full_name":"Josh Allen","age":28.0,"team":"BUF","number":17.0,"position":"QB","week_of_season":1,"predicted_fantasy_points":25.7600000092,"owner_name":"comebackking15","team_name":"Comeback \\ud83d\\udc51\\u2019s","is_user":false},{"id_sleeper":"4034","full_name":"Christian McCaffrey","age":27.0,"team":"SF","number":23.0,"position":"RB","week_of_season":10,"predicted_fantasy_points":24.9150000286,"owner_name":"jccarm","team_name":"DangeRUSS Last Ride","is_user":false},{"id_sleeper":"4034","full_name":"Christian McCaffrey","age":27.0,"team":"SF","number":23.0,"position":"RB","week_of_season":5,"predicted_fantasy_points":24.9150000286,"owner_name":"jccarm","team_name":"DangeRUSS Last Ride","is_user":false},{"id_slee

In [54]:
# Write to file
franchise_comparison.to_json('test_data/franchise_comparison.json', orient='records')